This is used for benchmark purposes, not a optimized pipeline for llm extraction

In [ ]:
from typing import List, Optional
from pydantic import BaseModel
import datetime
class Patent(BaseModel):
    title : str
    Application_Date: Optional[datetime.date]
    Publication_Date: Optional[datetime.date] 
    Applicants: list[str]
    Inventors: list[str]

    class Config:
        schema_extra = {
            "additionalProperties": False
        }
# serialize pydantic model into json schema
pydantic_schema = Patent.schema_json()
openai_api_key = 'API_KEY'
prompt = f"You are a helpful assistant that transform historical scans of patents to a JSON format. Make sure to get the dates and names correct. Take a second to think about your answer. Only output a json data, no explanation. Here's the json schema you must adhere to:\n{pydantic_schema}\n"

In [ ]:
import os
from pymongo import UpdateOne
from tqdm import tqdm
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

client = MongoClient("localhost", 29012)
db = client["test-database"]
collection_json = db["collection-json"]
def generate_query(item):
    text_clean = item['OCR'].decode('utf-8').replace('\n\n', ' ')
    return text_clean


In [ ]:
from tqdm import tqdm
import json
from openai import OpenAI
import os
count_token = 0
model = 'Llama-4-scout-17b-16e-instruct'
os.mkdir(f"/scratch/students/ndillenb/metadata/processing/llm/json_compare/{model}_json_compare/") if not os.path.exists(f"/scratch/students/ndillenb/metadata/processing/llm/json_compare/{model}_json_compare/") else None
for item in tqdm(list(collection_json.find({'Country':"US", 'OCR': {'$exists': True}, 'Title': {'$exists': True}, 'C_Application Date': {'$exists': True}, 'C_Publication Date': {'$exists': True}, 'clean_applicants': {'$exists': True}, 'clean_inventor': {'$exists': True}}).limit(100))):
    query = generate_query(item)
    openai_api_base = "https://api.lambda.ai/v1"

    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )

    count_token += len(f"{prompt}\n The patent text is: {query}\n\n")/4
    
    chat_completion = client.chat.completions.create(
    messages=[{
        "role": "system",
        "content": prompt
    }, {
        "role": "user",
        "content": f"The patent text is: {query}"
    }],
    model=model,
    )
    print('--response is--')
    print(chat_completion)
    json_llm = ''
    if "```json\n" in chat_completion.choices[0].message.content:
        try:
            response = chat_completion.choices[0].message.content[chat_completion.choices[0].message.content.find('```json\n'):chat_completion.choices[0].message.content.find('}```')]
            json_llm = response.strip('```json\n').strip('```')
            # Parse the JSON content into a Python dictionary
            json_llm = json.loads(json_llm)
        except:
            print('error parsing json')
            print(response)
            json_llm = None
    elif chat_completion.choices[0].message.content.startswith('{') or chat_completion.choices[0].message.content.replace('\n', '').replace("```", "").startswith('{'):
        response = chat_completion.choices[0].message.content
        try:
            json_llm = response.replace('\n', '').replace("```", "")
            # Parse the JSON content into a Python dictionary
            json_llm = json.loads(json_llm)
        except:
            print('error parsing json')
            print(response)
            json_llm = None
    # Store results for later evaluation
    with open(f"/scratch/students/ndillenb/metadata/processing/llm/json_compare/{model}_json_compare/json_llm_{item['_id']}.json", "w") as f:
        data = {'Title': item['Title'], 'Application_Date': item['C_Application Date'], 'Publication_Date': item['C_Publication Date'], 'Applicants': item['clean_applicants'], 'Inventors': item['clean_inventor']}
        # Convert datetime objects to strings for JSON serialization
        data_serializable = {
            key: (value.isoformat() if isinstance(value, datetime) else value)
            for key, value in data.items()
        }
        json.dump({'predicted': json_llm, 'expected': data_serializable}, f)
    print(f"Currently at about {count_token} tokens")